In [2]:
from confluent_kafka import Producer

def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

p = Producer({'bootstrap.servers': 'localhost:9092'})
try:
    p.produce('happiness-data', key='key', value='your_model_input_data', callback=acked)
except BufferError as e:
    print('Buffer full, waiting for free space on the queue')
p.poll(0)


0

In [3]:
from confluent_kafka import Consumer, KafkaError

c = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'mygroup',
    'auto.offset.reset': 'earliest'
})
c.subscribe(['happiness-data'])

try:
    while True:
        msg = c.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break
        data = msg.value().decode('utf-8')
        prediction = model.predict(data)  # Asumiendo que 'model' es tu modelo cargado
        print('Prediction:', prediction)
finally:
    c.close()


KeyboardInterrupt: 